In [4]:
import sys
import time
import json

from riotwatcher import LolWatcher, ApiError
import psycopg2

import match_fetcher

lol_watcher = LolWatcher('RGAPI-852cfdd4-e3af-40b0-b66d-d9ef1ca44c43')

In [6]:
my_region = 'na1'
region = 'na1'
account_id = lol_watcher.summoner.by_name(my_region, 'Corporate ShiIl')['puuid']
game_id = lol_watcher.match.matchlist_by_puuid('na1', account_id)[1]

In [15]:
import time

from riotwatcher import ApiError


def collect_match_data(region, account_id, game_id, lol_watcher):
    match_json = get_match_json(region, game_id, lol_watcher)

    # dictionary to hold all the data values of interest from the MatchDto
    data = {}

    data.update({'duration': match_json['info']['gameDuration']})

    # rest of the stats are found in the participants section
    participant_id = get_participant_id(match_json, account_id)
    # this shouldn't ever run but a -1 represents that the summoner was not in the match
    if participant_id == -1:
        print(game_id)
        return None

    # riot orders participants in the section 1-10, so can be indexed by subtracting one from the id
    participant_json = match_json['participants'][participant_id - 1]
    stats = participant_json['stats']

    data.update({
        'champion': participant_json['championId'],
        'win': stats['win'],
        'kills': stats['kills'],
        'deaths': stats['deaths'],
        'assists': stats['assists'],
        'gold_earned': stats['goldEarned'],
        'champion_damage': stats['totalDamageDealtToChampions'],
        'objective_damage': stats['damageDealtToObjectives'],
        'damage_healed': stats['totalHeal'],
        'vision_score': stats['visionScore'],
        'minions_killed': stats['totalMinionsKilled'],
        'neutral_monsters_killed': stats['neutralMinionsKilled'],
        'control_wards_purchased': stats['visionWardsBoughtInGame']
    })

    return data


# parses the participant ID based on the given summoner name
def get_participant_id(match_json, account_id):
    participants = match_json['participantIdentities']
    for summoner in participants:
        if summoner['player']['accountId'] == account_id:
            return summoner['participantId']

    return -1


def get_match_json(region, game_id, lol_watcher):
    try:
        return lol_watcher.match.by_id(region, game_id)
    except ApiError as err:
        if err.response.status_code == 429:
            print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        elif err.response.status_code == 404:
            print('Match not found for game ID: {}'.format(game_id))
        elif err.response.status_code == 504 or err.response.status_code == 503:
            # 504 happens randomly, wait a couple seconds then try again
            time.sleep(2)
            return get_match_json(region, game_id, lol_watcher)
        else:
            raise

In [11]:
match_json = get_match_json(region, game_id, lol_watcher)

In [25]:
match_json['info']['queueId']

450

In [24]:
match_json['info']['participants'][1]['summonerId']

'YPC0GKVY_59IDtj-z2UQ6w-TQStrQm7tfeCrrxHckMtFhfI'

In [16]:
collect_match_data(region, account_id, game_id, lol_watcher)

KeyError: 'participantIdentities'